In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_pickle("../grab-ai-safety-data/df_full.pickle")

In [3]:
df.head()

,bookingid,accuracy,bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,speed,label
9232561,0,12.0,143.298294,0.818112,-9.941461,-2.014999,-0.016245,-0.094040,0.070732,0.0,3.442991,0
10406491,0,8.0,143.298294,0.546405,-9.835590,-2.038925,-0.047092,-0.078874,0.043187,1.0,0.228454,0
1783254,0,8.0,143.298294,-1.706207,-9.270792,-1.209448,-0.028965,-0.032652,0.015390,2.0,0.228454,0
7886694,0,8.0,143.298294,-0.598145,-9.853534,-1.378574,-0.014297,-0.046206,0.021902,4.0,0.228454,0
13281304,0,8.0,143.298294,-0.608313,-9.539658,-1.794583,-0.007538,-0.023838,0.018068,5.0,0.228454,0


## Duration of trip
- This one is just obvious. Clear difference in average durations between safe and dangerous trips

In [8]:
df.groupby("bookingid").agg(
    {
        "second" : np.max,
        "label" : np.max
    }
).groupby("label").agg("mean")

,second
label,
0,265721.207147
1,1173.874625


## Acceleration
- Let's try euclidean first to see if that works
    - Mean of bookings, mean of label: no significant difference ~1% higher only
    - Max of the bookings, mean of label: ~12% higher for dangerous trips

In [47]:
df_acc = df.copy()
df_acc["acceleration"] = np.sqrt(
    (df_acc["acceleration_x"] ** 2) + (df_acc["acceleration_y"] ** 2) + (df_acc["acceleration_z"] ** 2)
)
test = df_acc.groupby("bookingid").agg(
    {
        "acceleration" : [np.max, np.mean],
        "label" : [np.max]
    }
)
test.columns = test.columns.get_level_values(0)
test.groupby("label").agg("mean")

,acceleration,acceleration
label,,
0,13.800160,9.872148
1,15.518975,9.904626


## Gyro
- Same technique with gyro to see
    - Max of bookings, mean of label: ~47% higher for dangerous trips
    - Mean of bookings, mean of label: ~36% higher for dangerous trips

In [51]:
df_gyro = df.copy()
df_gyro["gyro"] = np.sqrt(
    (df_gyro["gyro_x"] ** 2) + (df_gyro["gyro_y"] ** 2) + (df_gyro["gyro_z"] ** 2)
)
test = df_gyro.groupby("bookingid").agg(
    {
        "gyro" : [np.max, np.mean],
        "label" : [np.max]
    }
)
test.columns = test.columns.get_level_values(0)
test.groupby("label").agg("mean")

,gyro,gyro
label,,
0,1.380420,0.118911
1,2.041325,0.158067


## Speed
- More variations for speed

In [54]:
df_spd = df.copy()
test = df_spd.groupby("bookingid").agg(
    {
        "speed" : [np.max, np.mean],
        "label" : [np.max]
    }
)
test.columns = test.columns.get_level_values(0)
test.groupby("label").agg("mean")

,speed,speed
label,,
0,20.853004,8.606848
1,21.231855,7.885058
